In [10]:
import pandas as pd

# Load the dataset
file_path = 'C:\\Users\\HUCC\\Downloads\\cars_data_RAW (1).csv'
df = pd.read_csv(file_path)

# Rename columns for better readability
df.columns = [
    'Row_ID', 'Title', 'Model', 'Battery', 'Price_Range', '0-100_Acceleration',
    'Top_Speed', 'Range', 'Efficiency', 'Fastcharge', 'Germany_Price', 
    'Netherlands_Price', 'UK_Price', 'Drive_Configuration', 'Tow_Hitch', 
    'Towing_Capacity', 'Number_of_Seats'
]

# Clean the 'Range' column to ensure numeric values and convert from KM to MI
df['Range'] = df['Range'].str.extract('(\d+)').astype(float) * 0.621371

# Number of different models by each manufacturer
model_count = df.groupby('Title')['Model'].nunique()

# Average battery capacity for each manufacturer
avg_battery_capacity = df.groupby('Title')['Battery'].mean()

def clean_currency(value):
    if isinstance(value, str):
        value = value.replace('€', '').replace('£', '').replace(',', '').replace('*', '').strip()
        try:
            return float(value)
        except ValueError:
            return pd.NA
    elif isinstance(value,(int, float)):
        return value
    else:
        pd.NA


# Average cost for each manufacturer
df['UK_Price'] = df['UK_Price'].apply(clean_currency)

avg_cost = df.groupby('Title')['UK_Price'].mean()

# Average mileage (Range) for each manufacturer
avg_mileage = df.groupby('Title')['Range'].mean()

# Average miles per kWh (Effciency) for each manufacturer
mi_kwh = avg_mileage/avg_battery_capacity

# Effciency vs Cost
ef_v_cost = avg_cost/mi_kwh

# Combine all results into a single DataFrame for better readability
results = pd.DataFrame({
    'Model Count': model_count,
    'Average Battery Capacity': avg_battery_capacity,
    'Average Cost': avg_cost,
    'Average Mileage': avg_mileage,
    'Average Mi/kWh':mi_kwh,
    'Effciency vs Cost':ef_v_cost
}).reset_index()

results = results.dropna(subset=['Average Cost'])

#export the file for better more effcient visualisations in PBIX
file_path = "C:\\Users\\HUCC\\OneDrive - Direct Line Group\\Apprenticeship Work\\Term 3\\Data Science, M5, Data Professional Practice\\Public Data\\"
filname = "Electric Car Analysis.xlsx"

results.to_excel(file_path+filname,index=False)